In [1]:
import pandas as pd
import numpy as np
import joblib
import os
from sqlalchemy import create_engine
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from datetime import datetime

# --- CONFIGURACIÓN ---
MIN_SUPPORT = 0.01  # Soporte mínimo para itemsets frecuentes
MIN_LIFT = 1.0      # Lift mínimo para reglas de asociación
CARPETA_MODELOS = '../models'  # Carpeta donde guardar el modelo

# --- Detalles de conexión a tu base de datos PostgreSQL local ---
db_user = 'postgres'
db_password = 'postgres'
db_host = 'localhost'
db_port = '5433'
db_name = 'Tipvos' # Nombre de la base de datos

# Crear carpeta de modelos si no existe
os.makedirs(CARPETA_MODELOS, exist_ok=True)

# --- CONEXIÓN A POSTGRESQL LOCAL ---
# Construir la cadena de conexión
engine_string = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(engine_string)

print("=== ENTRENAMIENTO DE MODELO DE REGLAS DE ASOCIACIÓN ===")
print(f"📅 Fecha de entrenamiento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# --- CARGA Y PREPARACIÓN DE DATOS ---
print("\n--- 1. Cargando Datos ---")
try:
    df_ventas = pd.read_sql("SELECT * FROM fact_ventas", engine)
    df_productos = pd.read_sql("SELECT stockcode, descripcion FROM dim_producto", engine)
    print(f"✅ Datos cargados exitosamente de la base de datos local '{db_name}'.")
except Exception as e:
    print(f"❌ Error al cargar datos: {e}")
    print("Asegúrate de que la base de datos PostgreSQL local esté accesible y las tablas existan.")
    raise # Relanza la excepción para detener la ejecución si falla la carga

# Filtrar datos válidos
df_ventas = df_ventas[(df_ventas['cantidad'] > 0) & (df_ventas['precio_unitario'] > 0)]
df_ventas['fecha'] = pd.to_datetime(df_ventas['fecha'])

print(f"✅ Datos cargados: {len(df_ventas)} transacciones")
print(f"✅ Productos únicos: {df_ventas['stockcode'].nunique()}")
print(f"✅ Facturas únicas: {df_ventas['invoice'].nunique()}")

# --- 2. PREPARACIÓN DE DATOS PARA MARKET BASKET ANALYSIS ---
print("\n--- 2. Preparando Datos para Market Basket Analysis ---")

# Agrupar por factura para crear transacciones
transactions = df_ventas.groupby('invoice')['stockcode'].apply(list).tolist()

# Filtrar transacciones con al menos 2 productos
transactions_filtered = [trans for trans in transactions if len(trans) >= 2]

print(f"✅ Transacciones totales: {len(transactions)}")
print(f"✅ Transacciones con 2+ productos: {len(transactions_filtered)}")

# Codificar transacciones
te = TransactionEncoder()
te_ary = te.fit_transform(transactions_filtered)
basket_sets = pd.DataFrame(te_ary, columns=te.columns_)

print(f"✅ Matriz de transacciones creada: {basket_sets.shape}")

# --- 3. ENCONTRAR CONJUNTOS DE ARTÍCULOS FRECUENTES (APRIORI) ---
print("\n--- 3. Encontrar Conjuntos de Artículos Frecuentes (Apriori) ---")

frequent_itemsets = apriori(basket_sets, min_support=MIN_SUPPORT, use_colnames=True)

print(f"✅ Número de itemsets frecuentes encontrados: {len(frequent_itemsets)}")

if len(frequent_itemsets) > 0:
    print("\nPrimeros 10 itemsets frecuentes (ordenados por soporte):")
    print(frequent_itemsets.sort_values(by='support', ascending=False).head(10))
else:
    print("⚠️ No se encontraron itemsets frecuentes. Considera reducir min_support.")

# --- 4. GENERACIÓN DE REGLAS DE ASOCIACIÓN ---
print("\n--- 4. Generación de Reglas de Asociación ---")

if len(frequent_itemsets) > 1:
    # Generar las reglas de asociación
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=MIN_LIFT)

    if len(rules) > 0:
        # Ordenar las reglas
        rules = rules.sort_values(['lift', 'confidence'], ascending=[False, False])

        # Convertir frozensets a listas para mejor manejo
        rules['antecedents'] = rules['antecedents'].apply(lambda x: list(x))
        rules['consequents'] = rules['consequents'].apply(lambda x: list(x))

        print(f"✅ Número de reglas de asociación encontradas: {len(rules)}")
        print("\nTop 15 Reglas de Asociación (ordenadas por Lift y Confianza):")
        print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(15))

        # --- 5. PREPARAR MODELO PARA GUARDAR ---
        print("\n--- 5. Preparando Modelo para Guardar ---")
        
        # Crear diccionario con toda la información del modelo
        modelo_asociacion = {
            'fecha_entrenamiento': datetime.now(),
            'parametros': {
                'min_support': MIN_SUPPORT,
                'min_lift': MIN_LIFT
            },
            'estadisticas': {
                'total_transacciones': len(transactions),
                'transacciones_filtradas': len(transactions_filtered),
                'productos_unicos': df_ventas['stockcode'].nunique(),
                'itemsets_frecuentes': len(frequent_itemsets),
                'reglas_generadas': len(rules)
            },
            'frequent_itemsets': frequent_itemsets,
            'association_rules': rules,
            'transaction_encoder': te,
            'productos_info': df_productos  # Para obtener descripciones
        }

        # --- 6. GUARDAR MODELO ---
        print("\n--- 6. Guardando Modelo ---")
        modelo_path = os.path.join(CARPETA_MODELOS, 'market_basket_model.pkl')
        
        try:
            joblib.dump(modelo_asociacion, modelo_path)
            print(f"✅ Modelo guardado exitosamente en: {modelo_path}")
            
            # Guardar también un resumen legible
            resumen_path = os.path.join(CARPETA_MODELOS, 'market_basket_summary.txt')
            with open(resumen_path, 'w', encoding='utf-8') as f:
                f.write("=== RESUMEN DEL MODELO DE REGLAS DE ASOCIACIÓN ===\n")
                f.write(f"Fecha de entrenamiento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write(f"Parámetros: min_support={MIN_SUPPORT}, min_lift={MIN_LIFT}\n")
                f.write(f"Transacciones procesadas: {len(transactions_filtered)}\n")
                f.write(f"Itemsets frecuentes: {len(frequent_itemsets)}\n")
                f.write(f"Reglas de asociación: {len(rules)}\n\n")
                
                f.write("TOP 10 REGLAS DE ASOCIACIÓN:\n")
                f.write("="*50 + "\n")
                for idx, rule in rules.head(10).iterrows():
                    f.write(f"\nRegla {idx + 1}:\n")
                    f.write(f"   Si compra: {rule['antecedents']}\n")
                    f.write(f"   Entonces compra: {rule['consequents']}\n")
                    f.write(f"   Confianza: {rule['confidence']:.3f}\n")
                    f.write(f"   Lift: {rule['lift']:.3f}\n")
                    f.write(f"   Soporte: {rule['support']:.3f}\n")
            
            print(f"✅ Resumen guardado en: {resumen_path}")
            
        except Exception as e:
            print(f"❌ Error al guardar el modelo: {e}")

    else:
        print("⚠️ No se encontraron reglas de asociación. Considera ajustar los parámetros.")
        
else:
    print("⚠️ No hay suficientes itemsets frecuentes para generar reglas.")

# --- 7. RESUMEN FINAL ---
print(f"\n=== RESUMEN FINAL ===")
print(f"📊 Transacciones analizadas: {len(transactions_filtered)}")
print(f"🛍️ Productos únicos: {df_ventas['stockcode'].nunique()}")
print(f"📈 Itemsets frecuentes: {len(frequent_itemsets) if 'frequent_itemsets' in locals() else 0}")
print(f"🔗 Reglas de asociación: {len(rules) if 'rules' in locals() and len(rules) > 0 else 0}")
print(f"💾 Modelo guardado: {'SÍ' if 'modelo_asociacion' in locals() else 'NO'}")
print("🎉 Entrenamiento completado!")

=== ENTRENAMIENTO DE MODELO DE REGLAS DE ASOCIACIÓN ===
📅 Fecha de entrenamiento: 2025-07-28 12:24:26

--- 1. Cargando Datos ---
✅ Datos cargados exitosamente de la base de datos local 'Tipvos'.
✅ Datos cargados: 805549 transacciones
✅ Productos únicos: 4631
✅ Facturas únicas: 36969

--- 2. Preparando Datos para Market Basket Analysis ---
✅ Transacciones totales: 36969
✅ Transacciones con 2+ productos: 33995
✅ Matriz de transacciones creada: (33995, 4622)

--- 3. Encontrar Conjuntos de Artículos Frecuentes (Apriori) ---
✅ Número de itemsets frecuentes encontrados: 997

Primeros 10 itemsets frecuentes (ordenados por soporte):
      support  itemsets
612  0.142521  (85123A)
313  0.096102   (22423)
609  0.095014  (85099B)
584  0.077688   (84879)
21   0.075835   (20725)
75   0.073746   (21212)
524  0.060391   (47566)
304  0.058950   (22383)
23   0.058626   (20727)
303  0.056449   (22382)

--- 4. Generación de Reglas de Asociación ---
✅ Número de reglas de asociación encontradas: 1036

Top 

In [5]:
print("\n--- 5. Interpretación de las Reglas con Descripciones de Producto ---")

# Crear un diccionario para mapear stockcode a descripción
stockcode_to_desc = df_productos.drop_duplicates(subset=['stockcode']).set_index('stockcode')['descripcion'].to_dict()

# Función para reemplazar stockcodes por descripciones
def replace_with_descriptions(item_list):
    return [stockcode_to_desc.get(item, item) for item in item_list]

# Aplicar la función a las columnas 'antecedents' y 'consequents'
rules['antecedent_desc'] = rules['antecedents'].apply(replace_with_descriptions)
rules['consequent_desc'] = rules['consequents'].apply(replace_with_descriptions)

print("\nTop 15 Reglas de Asociación más Relevantes (con Descripciones de Producto):")
print(rules[['antecedent_desc', 'consequent_desc', 'support', 'confidence', 'lift']].head(15).to_string())

print("\n--- ¡Interpretación de las Reglas! ---")
print("Cada fila representa una regla: 'Si un cliente compra ANTECEDENTE, también es probable que compre CONSECUENTE'.")
print("- **Soporte (Support):** Frecuencia con la que ambos productos aparecen juntos. Un soporte de 0.01 significa que el 1% de todas las transacciones los contienen.")
print("- **Confianza (Confidence):** Probabilidad de que alguien que compra el antecedente, también compre el consecuente.")
print("  Ej: 0.60 significa que el 60% de las veces que se compra el antecedente, también se compra el consecuente.")
print("- **Lift:** Fuerza de la asociación. Un Lift > 1.0 es bueno. Cuanto mayor, más fuerte es la asociación y más interesante es la regla.")


--- 5. Interpretación de las Reglas con Descripciones de Producto ---

Top 15 Reglas de Asociación más Relevantes (con Descripciones de Producto):
                                                             antecedent_desc                                                          consequent_desc   support  confidence       lift
912   [WOODEN PICTURE FRAME WHITE FINISH, WOOD S/3 CABINET ANT WHITE FINISH]                                     [WOOD 2 DRAWER CABINET WHITE FINISH]  0.010903    0.725843  20.420099
913                                     [WOOD 2 DRAWER CABINET WHITE FINISH]   [WOODEN PICTURE FRAME WHITE FINISH, WOOD S/3 CABINET ANT WHITE FINISH]  0.010903    0.306743  20.420099
935         [WOODEN FRAME ANTIQUE WHITE , WOOD S/3 CABINET ANT WHITE FINISH]                                     [WOOD 2 DRAWER CABINET WHITE FINISH]  0.011173    0.695378  19.563041
938                                     [WOOD 2 DRAWER CABINET WHITE FINISH]         [WOODEN FRAME ANTIQUE WHITE , WOOD 